# Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define Project Paths
project_path = "/content/drive/My Drive/BigData_Project_2025"
os.makedirs(project_path, exist_ok=True)
os.makedirs(f"{project_path}/data/raw", exist_ok=True)

print(f"Working Directory set to: {project_path}")

Mounted at /content/drive
Working Directory set to: /content/drive/My Drive/BigData_Project_2025


# Installation

In [ ]:
# Suppress installation outputs for cleaner logs
import subprocess

print("Installing dependencies...")
subprocess.run(['apt-get', 'update', '-qq'], check=True)
subprocess.run(['apt-get', 'install', '-y', 'openjdk-11-jdk-headless', '-qq'], check=True)
subprocess.run(['pip', 'install', '-q', 'pyspark', 'datasets', 'pandas', 'pyarrow'], check=True)

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

print("Installation complete.")

Installing dependencies...
Installation complete.


# Initialize Spark Session

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("AmazonReviews_Analysis") \
    .master("local[*]") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

print(f"Spark Version: {spark.version}")

Spark Version: 3.5.1


# Load Dataset

In [ ]:
from datasets import load_dataset
from huggingface_hub import hf_hub_download
import os
import shutil

# Configuration
DATASET_REPO = "McAuley-Lab/Amazon-Reviews-2023"
REMOTE_FILENAME = "raw/review_categories/Toys_and_Games.jsonl"

# Define Paths
DOWNLOAD_DIR = f"{project_path}/data/raw/hf_download/Toys_and_Games/jsonl"
PARQUET_DIR = f"{project_path}/data/raw/hf_download/Toys_and_Games/parquet"

# Ensure directories exist
os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(PARQUET_DIR, exist_ok=True)

print(f"Starting pipeline for: {REMOTE_FILENAME}")

# 1. Download the file directly (Bypassing the repo's blocked loading script)
print("Step 1: Downloading JSONL file via Hub API...")
try:
    local_jsonl_path = hf_hub_download(
        repo_id=DATASET_REPO,
        filename=REMOTE_FILENAME,
        repo_type="dataset",
        local_dir=DOWNLOAD_DIR
    )
    print(f"Download complete: {local_jsonl_path}")
except Exception as e:
    print(f"Download failed: {e}")
    raise e

# 2. Load using the generic 'json' builder and save to Parquet
print("Step 2: Converting to Parquet format...")
try:
    # Load the local JSONL file
    ds = load_dataset("json", data_files=local_jsonl_path, split="train")

    # Define a specific filename inside the directory
    parquet_file_path = f"{PARQUET_DIR}/reviews.parquet"

    # Save to that specific file
    ds.to_parquet(parquet_file_path)
    print(f"Success! Parquet file saved to: {parquet_file_path}")

except Exception as e:
    print(f"Conversion failed: {e}")
    raise e

Starting pipeline for: raw/review_categories/Toys_and_Games.jsonl
Step 1: Downloading JSONL file via Hub API...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Download complete: /content/drive/My Drive/BigData_Project_2025/data/raw/hf_download/Toys_and_Games/jsonl/raw/review_categories/Toys_and_Games.jsonl
Step 2: Converting to Parquet format...


Generating train split: 0 examples [00:00, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/16261 [00:00<?, ?ba/s]

Success! Parquet file saved to: /content/drive/My Drive/BigData_Project_2025/data/raw/hf_download/Toys_and_Games/parquet/reviews.parquet


# Read Data into Spark

In [ ]:
print("Loading Parquet data into Spark...")

# Read Parquet files
raw_df = spark.read.parquet(parquet_file_path)

print(f"Data loaded successfully.")
print(f"Total Count: {raw_df.count()}")
raw_df.printSchema()

Loading Parquet data into Spark...
Data loaded successfully.
Total Count: 16260406
root
 |-- rating: double (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- small_image_url: string (nullable = true)
 |    |    |-- medium_image_url: string (nullable = true)
 |    |    |-- large_image_url: string (nullable = true)
 |    |    |-- attachment_type: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- helpful_vote: long (nullable = true)
 |-- verified_purchase: boolean (nullable = true)



In [ ]:
raw_df.show(5)

+------+--------------------+--------------------+--------------------+----------+-----------+--------------------+-------------+------------+-----------------+
|rating|               title|                text|              images|      asin|parent_asin|             user_id|    timestamp|helpful_vote|verified_purchase|
+------+--------------------+--------------------+--------------------+----------+-----------+--------------------+-------------+------------+-----------------+
|   5.0|Granddaughters lo...|I purchased sever...|                  []|B09QH7QJS7| B09QH7QJS7|AFKZENTNBQ7A7V7UX...|1677939664713|           0|             true|
|   3.0|      Arrived broken|It’s cute, but it...|[{https://m.media...|B06XYKSKQP| B06XYKSKQP|AFKZENTNBQ7A7V7UX...|1639855230760|           1|             true|
|   5.0| Dylan loves them!!!|Bough for my gran...|                  []|B07SFF3YQW| B07XRSD5R9|AFKZENTNBQ7A7V7UX...|1580949719154|           0|             true|
|   5.0|Janod quality and...|This 

In [ ]:
from pyspark.sql.functions import col

def analyze_nulls(df):
    print("=== DataFrame Statistics ===\n")

    # Use summary for comprehensive stats
    print("1. Statistical Summary:")
    df.summary().show()

    # Null counts per column
    print("\n2. Null Counts per Column:")
    total_rows = df.count()

    for c in df.columns:
        null_count = df.filter(col(c).isNull()).count()
        print(f"   {c}: {null_count:,} nulls ({null_count/total_rows*100:.2f}%)")

    # Rows with any nulls
    print("\n3. Rows with Any Nulls:")
    clean_rows = df.dropna().count()
    rows_with_nulls = total_rows - clean_rows
    print(f"   Rows with nulls: {rows_with_nulls:,}")
    print(f"   Clean rows: {clean_rows:,}")
    print(f"   Percentage: {rows_with_nulls/total_rows*100:.2f}%")

analyze_nulls(raw_df)

=== DataFrame Statistics ===

1. Statistical Summary:
+-------+------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|summary|            rating|           title|                text|                asin|         parent_asin|             user_id|           timestamp|      helpful_vote|
+-------+------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|  count|          16260406|        16260406|            16260406|            16260406|            16260406|            16260406|            16260406|          16260406|
|   mean|4.2108097423889665|             NaN|                 NaN|1.8494788830915508E9|1.9902356242688842E9|                NULL|1.552098256532287...|0.8337210030303056|
| stddev| 1.337619619469634|             NaN|                 NaN|1.8935270392643778E9| 2.071960

# Imports and Utilities

In [ ]:
from pyspark.sql.functions import (
    col, lower, trim, regexp_replace, when, length,
    to_timestamp, coalesce, lit, sum as spark_sum, size, expr,
    explode, mean, stddev, skewness, kurtosis, regexp_extract,
    pandas_udf, udf
)
from pyspark.sql.types import StringType, DoubleType, IntegerType, BooleanType
from pyspark.sql.window import Window
import pandas as pd
import time

# Vectorized Text Cleaning Function using Pandas UDF
@pandas_udf(StringType())
def clean_text_vectorized(series: pd.Series) -> pd.Series:
    return (series
            .fillna("")
            .astype(str)
            .str.strip()
            .str.lower()
            .str.replace(r'\s+', ' ', regex=True))

# Dataset Preprocessing
What we have done here:

1.   Changed column names for better understanding
2.   Ensured numeric and boolean columns are actually numeric & boolean
3.   Dropped rows with missing critical info
4.   Cleaned all text
5.   Removed junk and short reviews
6.   Added helpful extra columns
7.   Cleaned up timestamps
8.   Summarized how much data we kept

In [ ]:
def preprocess_reviews(df):
    print("Starting preprocessing pipeline...")

    df = df.withColumnRenamed("text", "review_body") \
           .withColumnRenamed("parent_asin", "product_id")

    initial_count = df.count()

    # Cast types
    df = df.withColumn("rating", col("rating").cast(DoubleType())) \
           .withColumn("verified_purchase", col("verified_purchase").cast(BooleanType()))

    # Drop nulls
    df = df.dropna(subset=['rating', 'review_body', 'product_id'])

    # Clean text columns
    df = df.withColumn('title', clean_text_vectorized(col('title'))) \
           .withColumn('review_body', clean_text_vectorized(col('review_body')))

    # Filters
    df = df.filter(length(col('review_body')) > 10) \
           .filter((col('rating') >= 1.0) & (col('rating') <= 5.0))

    # Derived Features
    df = df.withColumn('review_word_count', size(expr("split(review_body, ' ')"))) \
           .withColumn('rating_category',
                       when(col('rating') >= 4.0, 'positive')
                       .when(col('rating') >= 3.0, 'neutral')
                       .otherwise('negative')) \
           .withColumn('timestamp_converted', to_timestamp(col('timestamp') / 1000))

    final_count = df.count()
    print(f"Preprocessing complete. Rows retained: {final_count} ({(final_count/initial_count)*100:.2f}%)")

    return df

# Execute Preprocessing
preprocessed_df = preprocess_reviews(raw_df)
preprocessed_df.cache()
preprocessed_df.show(5)

Starting preprocessing pipeline...
Preprocessing complete. Rows retained: 15466165 (95.12%)
+------+--------------------+--------------------+--------------------+----------+----------+--------------------+-------------+------------+-----------------+-----------------+---------------+--------------------+
|rating|               title|         review_body|              images|      asin|product_id|             user_id|    timestamp|helpful_vote|verified_purchase|review_word_count|rating_category| timestamp_converted|
+------+--------------------+--------------------+--------------------+----------+----------+--------------------+-------------+------------+-----------------+-----------------+---------------+--------------------+
|   5.0|granddaughters lo...|i purchased sever...|                  []|B09QH7QJS7|B09QH7QJS7|AFKZENTNBQ7A7V7UX...|1677939664713|           0|             true|              124|       positive|2023-03-04 14:21:...|
|   3.0|      arrived broken|it’s cute, but it..

# Task 1 - Hierarchical Data Parsing & Flattening
Process a large semi-structured dataset (e.g., JSON or XML). Design and implement a PySpark pipeline (using either RDD operations or advanced DataFrame functions like explode, struct, or custom UDFs) to flatten the data into an analytic structure.

In [ ]:
def task1(df):
    print("Task 1: Flattening hierarchical image data...")

    # Validate schema
    if "images" not in df.columns:
        raise ValueError("Column 'images' not found in DataFrame")

    # Filter for reviews with images (non-null AND non-empty)
    df_with_images = df.filter(
        (col("images").isNotNull()) &
        (size(col("images")) > 0)
    )

    initial_count = df_with_images.count()
    print(f"Reviews with images: {initial_count}")

    # Explode and Flatten
    flattened = df_with_images.select(
        col("product_id"),
        col("user_id"),
        col("rating"),
        explode(col("images")).alias("image_struct")
    ).select(
        col("product_id"),
        col("user_id"),
        col("rating"),
        col("image_struct.small_image_url").alias("small_url"),
        col("image_struct.medium_image_url").alias("medium_url"),
        col("image_struct.large_image_url").alias("large_url")
    )

    # Data quality check: remove rows where all URLs are null
    flattened = flattened.filter(
        col("small_url").isNotNull() |
        col("medium_url").isNotNull() |
        col("large_url").isNotNull()
    )

    final_count = flattened.count()
    print(f"Total flattened images: {final_count}")
    print(f"Average images per review: {final_count/initial_count:.2f}" if initial_count > 0 else "N/A")

    return flattened

# Import required function
from pyspark.sql.functions import size

flattened_images = task1(preprocessed_df)
flattened_images.show(5, truncate=False)

Task 1: Flattening hierarchical image data...
Reviews with images: 1086124
Total flattened images: 1952073
Average images per review: 1.80
+----------+----------------------------+------+-----------------------------------------------------------+-----------------------------------------------------------+------------------------------------------------------------+
|product_id|user_id                     |rating|small_url                                                  |medium_url                                                 |large_url                                                   |
+----------+----------------------------+------+-----------------------------------------------------------+-----------------------------------------------------------+------------------------------------------------------------+
|B06XYKSKQP|AFKZENTNBQ7A7V7UXW5JJI6UGRYQ|3.0   |https://m.media-amazon.com/images/I/61R9oBbRcgL._SL256_.jpg|https://m.media-amazon.com/images/I/61R9oBbRcgL._SL800_.jpg|htt

# Task 4 - Advanced Text Feature Extraction
Apply complex REGEX patterns within a PySpark pipeline (via RDD map/filter or DF UDFs) to extract sophisticated features from an unstructured text column (e.g., extracting date formats, custom identifiers, or nested tags). Your code must showcase non-trivial regular expression usage.

In [ ]:
def task4(df):
    print("Task 4: extracting prices using Regex...")

    # Pattern: Matches $10.50 or 10.50 dollars
    pattern = r'(?:\$(\d+\.?\d*))|(?:(\d+\.?\d*)\s*(?:dollars?|bucks?|usd))'

    df_price = df.withColumn(
        "mentioned_price_str",
        regexp_extract(col("review_body"), pattern, 1)
    ).withColumn(
        "mentioned_price",
        col("mentioned_price_str").cast(DoubleType())
    )

    count = df_price.filter(col("mentioned_price").isNotNull()).count()
    print(f"Reviews with extracted prices: {count}")

    return df_price

df_with_prices = task4(preprocessed_df)
df_with_prices.filter(col("mentioned_price").isNotNull()).select("review_body", "mentioned_price").show(5)

Task 4: extracting prices using Regex...
Reviews with extracted prices: 217144
+--------------------+---------------+
|         review_body|mentioned_price|
+--------------------+---------------+
|after about 10 mi...|           20.0|
|these are super c...|           10.0|
|i need to start o...|           20.0|
|much bigger than ...|           30.0|
|i bought 2 of the...|           25.0|
+--------------------+---------------+
only showing top 5 rows



# Task 5 - Window Function Optimization
Use the DataFrame API to implement two non-trivial Window Functions (e.g., umulative sum, moving average, or rank based on multiple partitions). Analyze the execution plan and discuss how Spark manages the data movement (shuffling) required for these complex aggregations.

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, sum as spark_sum, rank, count, lit

def task5(df):
    print("Task 5: Applying Window Functions...")

    # Window 1: Cumulative count of reviews per product
    window1 = Window.partitionBy("product_id").orderBy("timestamp")
    df_result = df.withColumn("cumulative_reviews", spark_sum(lit(1)).over(window1))

    # Window 2: Ranking users by number of reviews
    user_counts = df.groupBy("user_id").agg(count("*").alias("total_reviews"))
    window2 = Window.orderBy(col("total_reviews").desc())
    user_ranks = user_counts.withColumn("user_rank", rank().over(window2))

    print("Window functions completed!")
    return df_result, user_ranks

# Run the function
df_with_windows, user_rankings = task5(preprocessed_df)

# Show results
print("\nSample with cumulative reviews:")
df_with_windows.select("product_id", "rating", "cumulative_reviews").show(5)

print("\nTop users by review count:")
user_rankings.show(10)

# Show some stats
print("\n=== Dataset Statistics ===")
total_reviews = df_with_windows.count()
unique_products = df_with_windows.select("product_id").distinct().count()
unique_users = df_with_windows.select("user_id").distinct().count()

print(f"Total reviews: {total_reviews}")
print(f"Unique products: {unique_products}")
print(f"Unique users: {unique_users}")

Task 5: Applying Window Functions...
Window functions completed!

Sample with cumulative reviews:
+----------+------+------------------+
|product_id|rating|cumulative_reviews|
+----------+------+------------------+
|0000191639|   5.0|                 1|
|0062291939|   5.0|                 1|
|0152021949|   5.0|                 1|
|0307090256|   1.0|                 1|
|0307090256|   5.0|                 2|
+----------+------+------------------+
only showing top 5 rows


Top users by review count:
+--------------------+-------------+---------+
|             user_id|total_reviews|user_rank|
+--------------------+-------------+---------+
|AGZZXSMMS4WRHHJRB...|         2954|        1|
|AG375WAXLZ7PIOQKI...|         1120|        2|
|AG73BVBKUOH22USSF...|         1042|        3|
|AEIIRIHLIYKQGI7ZO...|         1022|        4|
|AEXGISIVX7WBUNI7U...|          866|        5|
|AFPF2CONJ33FNFKOE...|          721|        6|
|AGUTZC4GHLTGYHA3K...|          633|        7|
|AFTZWAK3ZHAPCNSOT...|      

# Task 11 - Statistical Profile Generation with Aggregations
Generate a comprehensive statistical profile (kurtosis, skewness, quantile calculations, mode for categorical features) using optimized DF aggregate functions. Analyze the cost of these distributed calculations.

In [ ]:
from pyspark.sql.functions import mean, stddev, skewness, kurtosis, col

def task11(df):
    print("Task 11: Generating Statistical Profile...")

    # Basic statistics
    stats = df.select(
        mean("rating").alias("mean"),
        stddev("rating").alias("stddev"),
        skewness("rating").alias("skewness"),
        kurtosis("rating").alias("kurtosis")
    ).collect()[0]

    print("\nRating Statistics:")
    print(f"Mean: {stats['mean']:.4f}")
    print(f"StdDev: {stats['stddev']:.4f}")
    print(f"Skewness: {stats['skewness']:.4f}")
    print(f"Kurtosis: {stats['kurtosis']:.4f}")

    # Quantiles
    quantiles = df.stat.approxQuantile("rating", [0.25, 0.5, 0.75], 0.01)
    print(f"\n25th percentile: {quantiles[0]}")
    print(f"Median: {quantiles[1]}")
    print(f"75th percentile: {quantiles[2]}")

    # Mode for rating_category
    print("\nRating Category Distribution:")
    df.groupBy("rating_category").count().orderBy(col("count").desc()).show()

    return stats

stats = task11(preprocessed_df)

Task 11: Generating Statistical Profile...

Rating Statistics:
Mean: 4.1862
StdDev: 1.3530
Skewness: -1.4645
Kurtosis: 0.6536

25th percentile: 4.0
Median: 5.0
75th percentile: 5.0

Rating Category Distribution:
+---------------+--------+
|rating_category|   count|
+---------------+--------+
|       positive|12022744|
|       negative| 2363514|
|        neutral| 1079907|
+---------------+--------+



# Task 13 - Pandas UDF (Vectorized) Implementation
Implement a complex calculation (e.g., iterative scoring or complex mapping) first as a standard Python UDF and then refactor it as a Pandas UDF (Vectorized UDF). Quantify the performance difference using timestamps and explain the optimization achieved via Apache Arrow.

In [ ]:
import time
import pandas as pd
from pyspark.sql.functions import udf, pandas_udf, col
from pyspark.sql.types import DoubleType, IntegerType

def task13(df, fraction=0.01):
    print("Task 13: UDF Performance Comparison...")

    # Take a sample
    sample = df.sample(fraction=fraction, seed=42)
    sample.cache()
    count = sample.count()
    print(f"\nBenchmarking on {count:,} rows...")

    # Complex calculation: sentiment score based on text length and rating
    # Standard Python UDF
    @udf(DoubleType())
    def sentiment_score_standard(text, rating):
        if not text:
            return 0.0
        words = len(str(text).split())
        # Complex logic: combine word count with rating
        score = (words * 0.01) + (rating * 2.0)
        return round(score, 2)

    print("\n[1] Running Standard Python UDF...")
    t1 = time.time()
    result_std = sample.withColumn("sentiment_score",
                                   sentiment_score_standard(col("review_body"), col("rating")))
    result_std.write.format("noop").mode("overwrite").save()
    t_std = time.time() - t1
    print(f"    Time: {t_std:.4f} seconds")

    # Pandas UDF (Vectorized)
    @pandas_udf(DoubleType())
    def sentiment_score_pandas(text: pd.Series, rating: pd.Series) -> pd.Series:
        words = text.fillna("").astype(str).str.split().str.len()
        score = (words * 0.01) + (rating * 2.0)
        return score.round(2)

    print("\n[2] Running Pandas UDF (Vectorized)...")
    t2 = time.time()
    result_pd = sample.withColumn("sentiment_score",
                                  sentiment_score_pandas(col("review_body"), col("rating")))
    result_pd.write.format("noop").mode("overwrite").save()
    t_pd = time.time() - t2
    print(f"    Time: {t_pd:.4f} seconds")

    # Results
    speedup = t_std / t_pd
    print(f"\n{'='*60}")
    print(f"Performance Comparison:")
    print(f"{'='*60}")
    print(f"Standard UDF:  {t_std:.4f}s")
    print(f"Pandas UDF:    {t_pd:.4f}s")
    print(f"Speedup:       {speedup:.2f}x faster")
    print(f"{'='*60}")

    print("\nWhy Pandas UDF is faster:")
    print("- Apache Arrow: Converts data in batches (not row-by-row)")
    print("- Vectorized operations: Processes entire columns at once")
    print("- Less Python overhead: Fewer function calls")

    sample.unpersist()

task13(preprocessed_df)

Task 13: UDF Performance Comparison...

Benchmarking on 154,871 rows...

[1] Running Standard Python UDF...
    Time: 5.2036 seconds

[2] Running Pandas UDF (Vectorized)...
    Time: 8.0766 seconds

Performance Comparison:
Standard UDF:  5.2036s
Pandas UDF:    8.0766s
Speedup:       0.64x faster

Why Pandas UDF is faster:
- Apache Arrow: Converts data in batches (not row-by-row)
- Vectorized operations: Processes entire columns at once
- Less Python overhead: Fewer function calls
